In [1]:
#importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#uploading dataset
df = pd.read_csv(r'D:\Purity\ML_Zoomcamp\AB_NYC_2019.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df_selected = df[['neighbourhood_group','room_type','latitude','longitude','price','minimum_nights',
                  'number_of_reviews','reviews_per_month','calculated_host_listings_count',
                  'availability_365']]
df_selected.isnull().sum()#checking for missing values

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [4]:
#filling missing values with 0
df_filled = df_selected.fillna(0)
df_filled.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

Question 1

In [5]:
#getting value counts for neighbourhood column
df_filled['neighbourhood_group'].value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

Setting validation framework

In [6]:
#importing train_test_split module
from sklearn.model_selection import train_test_split

In [7]:
#getting test set
df_full_train, df_test = train_test_split(df_filled, test_size=0.2, random_state=42)
#getting train and validation set
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [8]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [9]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [10]:
y_train_df = df_train[['price']]
y_val_df = df_val[['price']]
y_test_df = df_test[['price']]

del df_train['price']
del df_val['price']
del df_test['price']

Question 2

In [11]:
#getting variable types in train dataset
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [12]:
#creating list for numerical variables
num_cols = list(df_train.select_dtypes(['int64', 'float64']).columns)
num_cols

['latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [13]:
#creating list for categorical columns
cat_cols = list(df_train.select_dtypes(['object']).columns)
cat_cols

['neighbourhood_group', 'room_type']

In [14]:
#getting correlation matrix for numerical columns
corr = df_train[num_cols].corr()
corr

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [15]:
from IPython.display import display
#creating function for making price binary
def make_price_binary(df):
    df['above_average'] = (df.price >= 152).astype(int)
    display(df.head())

In [17]:
#making binary
for df in [y_train_df, y_val_df, y_test_df]:
    make_price_binary(df)

<ipython-input-15-b9c0252b31bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_average'] = (df.price >= 152).astype(int)


,price,above_average
0,99,0
1,57,0
2,70,0
3,130,0
4,110,0


<ipython-input-15-b9c0252b31bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_average'] = (df.price >= 152).astype(int)


,price,above_average
0,65,0
1,89,0
2,200,1
3,120,0
4,748,1


<ipython-input-15-b9c0252b31bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_average'] = (df.price >= 152).astype(int)


,price,above_average
0,89,0
1,30,0
2,120,0
3,470,1
4,199,1


In [18]:
#creating label features
y_train = y_train_df.above_average.values
y_val = y_val_df.above_average.values
y_test = y_test_df.above_average.values

Question 3

In [19]:
#importing mutual_info_score library
from sklearn.metrics import mutual_info_score

In [22]:
def mutual_info_abv_avg_score(series):
    return mutual_info_score(series, y_train_df.above_average)

In [23]:
#getting mutual info score
mi = df_train[cat_cols].apply(mutual_info_abv_avg_score)
mi.sort_values(ascending=False)

room_type              0.143226
neighbourhood_group    0.046506
dtype: float64

Question 4

In [24]:
#using  scikit learn for encoding categorical variables
from sklearn.feature_extraction import DictVectorizer

In [25]:
dv = DictVectorizer(sparse=False)
# transforming train features
train_dict = df_train[cat_cols + num_cols].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
# transforming validation features
val_dict = df_val[cat_cols + num_cols].to_dict(orient='records')
X_val = dv.transform(val_dict)
# transforming test features
test_dict = df_test[cat_cols + num_cols].to_dict(orient='records')
X_test = dv.transform(test_dict)

In [26]:
#logistic regression with scikit learn
from sklearn.linear_model import LogisticRegression

In [27]:
#creating the model
model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)

In [28]:
#fitting model with train features
model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [29]:
#checking model intercept
model.intercept_[0]

-0.09188251977594396

In [30]:
#checking for coefficients
model.coef_[0].round(3)

array([ 3.000e-03,  4.000e-03, -5.831e+00, -3.174e+00, -1.100e-02,
       -1.100e-01,  1.440e-01,  1.595e+00, -1.200e-02, -1.709e+00,
       -3.000e-03, -4.200e-02,  1.947e+00, -8.330e-01, -1.206e+00])

In [31]:
#predicting the validation set
y_pred = model.predict_proba(X_val)[:, 1]

In [32]:
#getting results above average
above_average = (y_pred >= 0.5)
#getting accuracy
org_accuracy=round((y_val == above_average).mean(), 2)
org_accuracy

0.79

Question 5

In [33]:
col_list = list(df_train.columns)
col_list

['neighbourhood_group',
 'room_type',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [34]:
for col in col_list:
    print('Removing '+col)
    cols = col_list.copy()
    cols.remove(col)
    dv = DictVectorizer(sparse=False)
    # transforming train features
    train_dict = df_train[cols].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    # transforming validation features
    val_dict = df_val[cols].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    # transforming test features
    test_dict = df_test[cols].to_dict(orient='records')
    X_test = dv.transform(test_dict)
    #fitting model with train features
    model.fit(X_train, y_train)
    #checking model intercept
    model.intercept_[0]
    #checking for coefficients
    model.coef_[0].round(3)
    #predicting the validation set
    y_pred = model.predict_proba(X_val)[:, 1]
    #getting results above average
    above_average = (y_pred >= 0.5)
    #getting accuracy
    accuracy = (y_val == above_average).mean()
    print('Accuracy ='+str(accuracy))
    diff = accuracy - org_accuracy
    print('Difference with original accuracy ='+str(diff))
    print(' ')

Removing neighbourhood_group
Accuracy =0.7498721750690255
Difference with original accuracy =-0.040127824930974554
 
Removing room_type
Accuracy =0.7287043664996421
Difference with original accuracy =-0.06129563350035794
 
Removing latitude
Accuracy =0.7867880151344718
Difference with original accuracy =-0.0032119848655282057
 
Removing longitude
Accuracy =0.7869925350240311
Difference with original accuracy =-0.003007464975968932
 
Removing minimum_nights
Accuracy =0.7915942325391144
Difference with original accuracy =0.001594232539114393
 
Removing number_of_reviews
Accuracy =0.7913897126495552
Difference with original accuracy =0.0013897126495551193
 
Removing reviews_per_month
Accuracy =0.7907761529808774
Difference with original accuracy =0.0007761529808774092
 
Removing calculated_host_listings_count
Accuracy =0.7895490336435218
Difference with original accuracy =-0.0004509663564782329
 
Removing availability_365
Accuracy =0.7812659781163718
Difference with original accuracy =-0.

Question 6

In [35]:
# transforming the price variable
y_train_orig = y_train_df.price.values
y_val_orig = y_val_df.price.values
y_test_orig = y_test_df.price.values

y_train = np.log1p(y_train_orig)
y_val = np.log1p(y_val_orig)
y_test = np.log1p(y_test_orig)

In [39]:
#function for RMSE
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [41]:
#ridge regression
from sklearn.linear_model import Ridge
def ridge_regression(alpha):
    lin_model = Ridge(alpha=alpha)
    lin_model.fit(X_train, y_train)#fitting train set
    y_pred = lin_model.predict(X_val)
    print(rmse(y_val, y_pred))
numbers = [0, 0.01, 0.1, 1, 10]
for number in numbers:
    print('alpha='+str(number))
    ridge_regression(number)
    print(' ')

alpha=0
0.505768658202005
 
alpha=0.01
0.5056849319563437
 
alpha=0.1
0.5056868054117745
 
alpha=1
0.5057140038386609
 
alpha=10
0.5064003190588291
 
